In [1]:
import json
from urllib.request import urlopen
import arcpy
from arcpy import env
import os

In [2]:
arcpy.env.overwriteOutput = True

In [3]:
# function to convert json from url to features
def json2shp(url_string, json_folder, geodatabase, filename):
    response = urlopen(url_string)
    data = response.read()
    txt_str = data.decode('utf-8')
    lines = txt_str.split("\r\n")
    path = json_folder + '/' + filename + '.json'
    fx = open(path, "w")
    for line in lines:
        fx.write(line+ "\n")
    fx.close()
    arcpy.JSONToFeatures_conversion(path, os.path.join(geodatabase, filename))

In [4]:
url_string = "https://services.arcgis.com/uUvqNMGPm7axC2dD/arcgis/rest/services/oregondot_its_facilities_and_signals/FeatureServer/3/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [5]:
json_folder = "//clsrv111.int.lcog.org/transpor/MPO/RTP/FY20 2045 Update/Data and Resources/Data"

In [6]:
filename = "ODOT_traffic_signals"

In [7]:
geodatabase = os.path.join(json_folder, "StoryMap/StoryMap.gdb")

In [8]:
json2shp(url_string, json_folder, geodatabase, filename)

In [9]:
url_string = "https://services6.arcgis.com/5YJfprDR6T2FT4tY/arcgis/rest/services/Traffic_Infrastructure_View/FeatureServer/2?f=pjson"

In [10]:
filename = "SPR_traffic_signals"

In [11]:
json2shp(url_string, json_folder, geodatabase, filename)

ExecuteError: ERROR 001558: Error parsing json file '//clsrv111.int.lcog.org/transpor/MPO/RTP/FY20 2045 Update/Data and Resources/Data/SPR_traffic_signals.json'. 
Failed to execute (JSONToFeatures).


In [12]:
path = json_folder + '/' + filename + '.json'

In [14]:
arcpy.JSONToFeatures_conversion(path, os.path.join(geodatabase, filename))

ExecuteError: ERROR 001558: Error parsing json file '//clsrv111.int.lcog.org/transpor/MPO/RTP/FY20 2045 Update/Data and Resources/Data/SPR_traffic_signals.json'. 
Failed to execute (JSONToFeatures).
